# About Crim Intervals Pattern Match for One Piece

## Notes about the Various Parameters

- *Length of the Soggetto*: into_patterns([vectors.semitone_intervals], 5) The **number** in this command represents the **minimum number of vectors to find**. 5 vectors is 6 notes.

### Chromatic vs Diatonic 
- *Chromatic* uses `into_patterns([vectors.semitone_intervals], 5)`
- *Diatonic* uses `into_patterns([vectors.generic_intervals], 5)`

### Exact vs Close  
- Exact is exact in *all* ways find_exact_matches(patterns, 2). The **number** in this command represents the **minimum number of matching melodies needed before reporting**. This allows us to filter for common or uncommon soggetti.
- Close matches allow for melodic variation (see more below). `find_close_matches(patterns, 2, 1)`
. The **first number** in this command is the minimum number of melodies needed before reporting; the **second number** is threshold needed in order to find a match. Lower number = very similar; higher number = less similar

### More about Close Matches  
- The threshold for close matches is determined by the third number called in the method. We select two patterns, then compare *each vector in each pattern successively*. The "differences" between each vector are summed. If that value is below the threshold specified, we consider the two patterns closely matched.
- The format of the method call is  `find_close_matches(the array you get from into_patterns, minimum matches needed to be displayed, threshold for close match)`.

### About Rhythmic Durations

For `find_close_matches` and `find_exact_matches`, rhythmic variation/duration is displayed, but not factored into the calculation.

## How to Enter Parameters

- Code for relatively easy interaction with the various parameters.  
- The logic 'reads' the choices indicated in the initial list, then runs the appropriate sub-routine.

### Setting Parameters

- interval type (generic or semitone)
- match type (exact or close)
- vector size (vectors + 1 = number of notes in the soggetto)
- minimum matches (how many matched soggetti needed in order to report)
- close distance (the maximum edit distance to consider for reporting in the case of 'close' matches)

               `interval_type="generic", 
               match_type="exact", 
               vector_size=5, 
               min_matches=2, 
               close_distance=2)`
               


### Dataframe Preview vs Full Results in Browser

- Comment out the first of these for preview only.  Include for full results

    - `#pd.set_option("display.max_rows", None, "display.max_columns", None)`
    - `return pd.DataFrame(match_data)`
    
### To Set CSV Output

- Specify file name here `pd.Series(match_data).to_csv("Match_data_10_13.csv")`

In [30]:
import crim_intervals as ci
import pandas as pd

def get_match_data_for_piece(piece_id, 
                             interval_type="generic", 
                             match_type="exact", 
                             vector_size=5, 
                             min_matches=2, 
                             close_distance=2):
    
    url = f"https://crimproject.org/mei/{piece_id}.mei"
    score = ci.ScoreBase(url)
    vectors = ci.IntervalBase(score.note_list)
    
    if interval_type == "generic":
        patterns = ci.into_patterns([vectors.generic_intervals], vector_size)
    elif interval_type == "semitone":
        patterns = ci.into_patterns([vectors.semitone_intervals], vector_size)
    else:
        raise Exception("Interval type must be 'generic' or 'semitone'")
        
    if match_type == "exact":
        matches = ci.find_exact_matches(patterns, min_matches)
    elif match_type == "close":
        matches = ci.find_close_matches(patterns, min_matches, close_distance)
    else:
        raise Exception("Match type must be 'exact' or 'close'")
    
    match_data = []
    
    for match_series in matches:
        for match in match_series.matches:
            match_dict = {
              "pattern_generating_match": match_series.pattern,
              "pattern_matched": match.pattern, 
              "piece_title": match.first_note.metadata.title, 
              "part": match.first_note.part, 
              "start_measure": match.first_note.note.measureNumber, 
              "end_measure": match.last_note.note.measureNumber, 
              "note_durations": match.durations, 
              "ema": match.ema, 
              "ema_url": match.ema_url
            }

            match_data.append(match_dict)
        
    pd.Series(match_data).to_csv("Match_data_10_13.csv")        
    #pd.set_option("display.max_rows", None, "display.max_columns", None)
    return pd.DataFrame(match_data)
    




## Select Piece ID

- Here we provide the name of the CRIM Piece, such as CRIM_Model_0001, in this format:

    `get_match_data_for_piece('CRIM_Model_0008')`

In [31]:
get_match_data_for_piece('CRIM_Model_0002')




Requesting file from https://crimproject.org/mei/CRIM_Model_0002.mei...
Successfully imported.
Finding exact matches...
76 melodic intervals had more than 2 exact matches.



/Users/rfreedma/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:47: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.


,pattern_generating_match,pattern_matched,piece_title,part,start_measure,end_measure,note_durations,ema,ema_url
0,"[1, 1, 2, 2, 3]","[1, 1, 2, 2, 3]",O gente brunette,Superius,1,1,"[1.0, 1.0, 1.0, 1.0, 2.0, 2.0]","1-1/1/@1.0-end,@start-3.0",https://ema.crimproject.org/https%3A%2F%2Fcrim...
1,"[1, 1, 2, 2, 3]","[1, 1, 2, 2, 3]",O gente brunette,Superius,6,7,"[1.0, 1.0, 1.0, 1.0, 2.0, 2.0]","6-7/1/@1.0-end,@start-3.0",https://ema.crimproject.org/https%3A%2F%2Fcrim...
2,"[1, 1, 2, 2, 3]","[1, 1, 2, 2, 3]",O gente brunette,Superius,24,24,"[1.0, 1.0, 1.0, 1.0, 2.0, 2.0]","24-24/1/@1.0-end,@start-3.0",https://ema.crimproject.org/https%3A%2F%2Fcrim...
3,"[1, 1, 2, 2, 3]","[1, 1, 2, 2, 3]",O gente brunette,Superius,29,30,"[1.0, 1.0, 1.0, 1.0, 2.0, 2.0]","29-30/1/@1.0-end,@start-3.0",https://ema.crimproject.org/https%3A%2F%2Fcrim...
4,"[1, 2, 2, 3, -2]","[1, 2, 2, 3, -2]",O gente brunette,Superius,1,2,"[1.0, 1.0, 1.0, 2.0, 2.0, 2.0]","1-2/1/@2.0-end,@start-1.0",https://ema.crimproject.org/https%3A%2F%2Fcrim...
...,...,...,...,...,...,...,...,...,...
310,"[-5, 4, -2, 2, -3]","[-5, 4, -2, 2, -3]",O gente brunette,Bassus,23,24,"[2.0, 4.0, 1.0, 1.0, 1.0, 1.0]","23-24/4/@3.0-end,@start-4.0",https://ema.crimproject.org/https%3A%2F%2Fcrim...
311,"[-5, 4, -2, 2, -3]","[-5, 4, -2, 2, -3]",O gente brunette,Bassus,28,29,"[4.0, 4.0, 1.0, 1.0, 1.0, 1.0]","28-29/4/@1.0-end,@start-4.0",https://ema.crimproject.org/https%3A%2F%2Fcrim...
312,"[4, -2, 2, -3, -2]","[4, -2, 2, -3, -2]",O gente brunette,Bassus,6,7,"[4.0, 1.0, 1.0, 1.0, 1.0, 2.0]","6-7/4/@1.0-end,@start-1.0",https://ema.crimproject.org/https%3A%2F%2Fcrim...
313,"[4, -2, 2, -3, -2]","[4, -2, 2, -3, -2]",O gente brunette,Bassus,23,24,"[4.0, 1.0, 1.0, 1.0, 1.0, 2.0]","23-24/4/@1.0-end,@start-1.0",https://ema.crimproject.org/https%3A%2F%2Fcrim...
